In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# make testing data
N = 1000
ts_series = np.sin(0.1*np.arange(N))
ts_series_with_noise = np.sin(0.1*np.arange(N)) + np.random.randn(N) * 0.1


# Plot the data
plt.rcParams.update({'figure.figsize':(12,5), 'figure.dpi':100})
fig, ax = plt.subplots(1, 2)
plt.figure(figsize=(10, 5))
ax[0].plot(ts_series)
ax[1].plot(ts_series_with_noise)
plt.show();

#### Построим датасет из синтетических данных для иллюстрации прогнозирования временного ряда:

In [ ]:
# Let's experiment if we can use T past values to predict the next value (values)
T = 10
X = []
Y = []

for val in range(len(ts_series) - T):
    x = ts_series[val:val+T] # 0:10, 1:11, 2:12, 3:13, 4:14, 5:15...
    X.append(x)
    y = ts_series[val+T] # 0:10 constant
    Y.append(y)

# Reshaping formed array to ensure the valid dims
X = np.array(X).reshape(-1, T) # any number of rows and only with 10 number of feature-columns
Y = np.array(Y).reshape(-1, 1) # any number of rows and only 1 dim feature-vector
N = len(X)
print("X shape: {}  Y shape: {} and N = {}".format(X.shape, Y.shape, N))

In [ ]:
X[:5]

In [ ]:
Y[:5]

### Build Autoregressive linear model:

In [ ]:
model = torch.nn.Linear(T, 1)

# Loss and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

# Make inputs and targets in the form of tensor
X_train = torch.from_numpy(X[:-N//2].astype(np.float32))
y_train = torch.from_numpy(Y[:-N//2].astype(np.float32))

X_test = torch.from_numpy(X[-N//2:].astype(np.float32))
y_test = torch.from_numpy(Y[-N//2:].astype(np.float32))

In [ ]:
# Training process
def full_GD(model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs=200):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)

    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)

        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        train_losses[epoch] = loss.item()

        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)
        test_losses[epoch] = test_loss.item()

        print(f"Epoch: {epoch+1}/{epochs}       Train Loss: {loss.item():.4f}      Test Loss: {test_loss.item():.4f}")
    return train_losses, test_losses

In [ ]:
train_losses, test_losses = full_GD(model, criterion, optimizer, X_train, y_train, X_test, y_test)

In [ ]:
plt.plot(train_losses,label="train loss")
plt.plot(test_losses, label="test loss")
plt.legend()
plt.show();

In [ ]:
# Wrong forecast using true targets
validation_target = Y[-N//2:]
validation_prediction = []

# index of 1st validation input
i = 0

while len(validation_prediction) < len(validation_target):
    input_ = X_test[i].view(1, -1)
    p = model(input_)[0, 0].item() # 1x1 array -> scalar
    i+=1

    validation_prediction.append(p)

In [ ]:
plt.plot(validation_target, label='forecast target')
plt.plot(validation_prediction, label='forecast predictions')
plt.legend()

In [ ]:
# Forecast future values (use only self-predictions for making future predictions)

validation_target = Y[-N//2:]
validation_predictions = []

# last train input
# 1-D array of length T
last_x = torch.from_numpy(X[-N//2].astype(np.float32))

while len(validation_predictions) < len(validation_target):
  input_ = last_x.view(1, -1)
  p = model(input_)
  # [0,0] # 1x1 array -> scalar

  # update the predictions list
  validation_predictions.append(p[0,0].item())

  # make the new input
  last_x = torch.cat((last_x[1:], p[0]))

In [ ]:
plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()